In [1]:
import datetime as datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


plt.style.use('default') # Make the graphs a bit prettier
plt.rcParams['figure.figsize'] = (15, 5)

In [2]:
#cargamos todos los archivos de la pagina que se encuentren dentro de la carpeta enUso
import os
dsSell = pd.DataFrame()
for filename in os.listdir("datosCompletos/propieades/enUso"):
    dataSet = pd.read_csv("datosCompletos/propieades/enUso/" + filename, low_memory=False)
    dsSell = dsSell.append(dataSet)

In [3]:
dsSell.head()

,country_name,created_on,currency,description,expenses,extra,floor,geonames_id,id,image_thumbnail,...,price_per_m2,price_usd_per_m2,properati_url,property_type,rooms,state_name,surface_covered_in_m2,surface_in_m2,surface_total_in_m2,title
0,NaN,2012-08-08,NaN,NaN,NaN,NaN,NaN,3429308.0,NaN,http://cf-thumbs.properati.com.ar/yELs9sxKz88c...,...,NaN,NaN,http://www.properati.com.ar/9h_venta_casa_san-...,house,3.0,NaN,NaN,NaN,NaN,NaN
1,NaN,2012-08-08,NaN,NaN,850 aprox,NaN,10.0,3436077.0,NaN,http://cf-thumbs.properati.com.ar/1RXv7KDah9Yt...,...,NaN,NaN,http://www.properati.com.ar/ag_venta_departame...,apartment,3.0,NaN,NaN,102.0,NaN,NaN
2,NaN,2012-08-08,NaN,NaN,NaN,NaN,11.0,3427467.0,NaN,http://cf-thumbs.properati.com.ar/Mkt0XSfl3IEb...,...,NaN,NaN,http://www.properati.com.ar/ah_venta_departame...,apartment,3.0,NaN,NaN,73.0,NaN,NaN
3,NaN,2012-08-08,NaN,NaN,NaN,NaN,NaN,3427467.0,NaN,http://cf-thumbs.properati.com.ar/VBY8U6EZL4ru...,...,NaN,NaN,http://www.properati.com.ar/aj_venta_casa_vill...,house,4.0,NaN,NaN,485.0,NaN,NaN
4,NaN,2012-08-08,USD,NaN,NaN,NaN,NaN,3427467.0,NaN,http://cf-thumbs.properati.com.ar/WYxr8Dmu1rzj...,...,NaN,315.186246,http://www.properati.com.ar/ak_venta_casa_vill...,house,4.0,NaN,NaN,349.0,NaN,NaN


In [4]:
#Queremos quedarnos con la zona, el municipio y los que esten en CABA y GBA
def getMunicipioYZona(df, dataSet):
    df['municipio'] = df['place_with_parent_names'].apply(lambda x: x.split('|')[3])
    df['zona'] = df['place_with_parent_names'].apply(lambda x: x.split('|')[2])

    df['filtro'] = df['zona'].apply(lambda x: ((x=='Capital Federal')or(x=='Bs.As. G.B.A. Zona Norte')or(x=='Bs.As. G.B.A. Zona Sur')or(x=='Bs.As. G.B.A. Zona Oeste')))
    if(dataSet == 'train'):
        return df[df.filtro]
    else:
        return df
    

In [5]:
#Como nos importa mucho el precio con respecto a otras cosas vamos a sacar a los que tienen esa columna en NaN
def filtrarPrecioNulo(df):
    df['tiene_precio'] = df['price'].apply(lambda x: pd.notnull(x))
    return df[df.tiene_precio]

In [6]:
#convertimos el formato de la fecha
def formatearFecha(df):
    df['year'] =pd.to_numeric(df['created_on'].map( lambda x: x.split('-')[0]))
    df['month'] = pd.to_numeric(df['created_on'].map( lambda x: x.split('-')[1]))
    df['created_on'] = df['created_on'] .map(lambda x: x.split('-')[0]+x.split('-')[1]+x.split('-')[2])


In [7]:
def getInfoFromDescription(df):
    df['cochera'] = df.description.str.contains('cochera')
    df['pileta'] = df.description.str.contains('pileta')
    df['sum'] = df.description.str.contains('sum')
    df['aire acondicionado'] = df.description.str.contains('aire acondicionado')
    df['calefaccion'] = df.description.str.contains('calefaccion')
    df['seguridad'] = df.description.str.contains('seguridad')
    df['estrenar'] = df.description.str.contains('estrenar')
    df['emprendimiento'] = df.description.str.contains('emprendimiento')

    #convertimos los valores false , true y nan de las busquedas anteriores

    df['cochera'] = df['cochera'].map(lambda x: 0 if x == 'False' else 1 if x == 'True' else 0.5)
    df['pileta'] = df['pileta'].map(lambda x: 0 if x == 'False' else 1 if x == 'True' else 0.5)
    df['sum'] = df['sum'].map(lambda x: 0 if x == 'False' else 1 if x == 'True' else 0.5)
    df['aire acondicionado'] = df['aire acondicionado'].map(lambda x: 0 if x == 'False' else 1 if x == 'True' else 0.5)
    df['calefaccion'] = df['calefaccion'].map(lambda x: 0 if x == 'False' else 1 if x == 'True' else 0.5)
    df['seguridad'] = df['seguridad'].map(lambda x: 0 if x == 'False' else 1 if x == 'True' else 0.5)
    df['estrenar'] = df['estrenar'].map(lambda x: 0 if x == 'False' else 1 if x == 'True' else 0.5)
    df['emprendimiento'] = df['emprendimiento'].map(lambda x: 0 if x == 'False' else 1 if x == 'True' else 0.5)
    

In [8]:
def getPrecioPorMetroCuadradoPorZona(df,df2, columna):
    df.drop('price_usd_per_m2', axis=1, inplace=True)
    return pd.merge(df, df2, on=columna)
    

In [9]:
#pasar todas las superficies de surface_in_m2 a total_surface_in_m2
def estandarizarCampoSuperfice(df):
    mask = (df['surface_total_in_m2'].isnull())
    df['surface_total_in_m2'][mask] = df['surface_in_m2'][mask]
    mask = (df['surface_covered_in_m2'].isnull())
    df['surface_covered_in_m2'][mask] = df['surface_in_m2'][mask]

In [10]:
#drop de columnas
def dropColumnas(df):
    df.drop('price_per_m2', axis = 1, inplace = True)
    df.drop('place_with_parent_names', axis = 1, inplace = True)
    df.drop('created_on', axis = 1, inplace = True)
    df.drop('description', axis = 1, inplace = True)
    df.drop('currency', axis=1, inplace=True)
    df.drop('extra', axis=1, inplace=True)
    df.drop('id', axis=1, inplace=True)
    df.drop('image_thumbnail', axis=1, inplace=True)
    df.drop('properati_url', axis=1, inplace=True)
    df.drop('title', axis=1, inplace=True)
    df.drop('lat-lon', axis=1, inplace=True)
    df.drop('floor', axis=1, inplace=True)
    df.drop('operation', axis=1, inplace=True)
    df.drop('price', axis=1, inplace=True)
    df.drop('price_aprox_local_currency', axis=1, inplace=True)
    df.drop('country_name', axis=1, inplace=True)
    df.drop('place_name', axis=1, inplace=True)
    df.drop('municipio', axis=1, inplace=True)
    df.drop('state_name', axis=1, inplace=True)
    df.drop('geonames_id', axis=1, inplace=True)
    df.drop('surface_in_m2', axis=1, inplace=True)
    df.drop('filtro', axis=1, inplace=True)
    df.drop('tiene_precio', axis=1, inplace=True)

In [11]:
dsSell.columns

Index(['country_name', 'created_on', 'currency', 'description', 'expenses',
       'extra', 'floor', 'geonames_id', 'id', 'image_thumbnail', 'lat',
       'lat-lon', 'lon', 'operation', 'place_name', 'place_with_parent_names',
       'price', 'price_aprox_local_currency', 'price_aprox_usd',
       'price_per_m2', 'price_usd_per_m2', 'properati_url', 'property_type',
       'rooms', 'state_name', 'surface_covered_in_m2', 'surface_in_m2',
       'surface_total_in_m2', 'title'],
      dtype='object')

In [12]:
#preprocessing

def preprocesamientoExpensas(df):
    #eliminamos todo lo q no es numero de expenses
    df['expenses'] = df['expenses'].str.extract('(\d+)')
    #convertimos a float

    df['expenses'] =pd.to_numeric(df.expenses, errors='coerce').notnull()
    

In [13]:
#one hot encoding de los place name. me suena mal muchas columnas. performance
def oneHotEncoding(df, columna):
  
    one_hot = pd.get_dummies(df[columna])
    df.drop(columna, axis=1, inplace=True)
    df = pd.concat([df, one_hot], axis=1)
    return df
    

In [14]:
#convertir lat, long a coordenadas x,y 
import pyproj as proj

def latLongToXY(long, lat):
    crs_wgs = proj.Proj(init='epsg:4326') # assuming you're using WGS84 geographic
    crs_bng = proj.Proj(init='epsg:27700') # use a locally appropriate projected CRS

    x, y = proj.transform(crs_wgs, crs_bng,long, lat)
    return x,y

In [15]:
def calcularEstaciones(reg):
    cantidad = 0
    for index, estacion in estacionesSubte.iterrows():
        cantidad = cantidad + 1 if (distanciaLatLong(reg[3], reg[2],estacion['X'], estacion['Y'])) < 0.4 else 0
    
    return cantidad
        

In [16]:
#cargamos los datos de las localizaciones de transporte y agregamos a dataset (por ahora solo cuenta la cantidad de estaciones)
#datos sacados de data.buenosaires.gob.ar
estacionesSubte = pd.read_csv("datosCompletos/transporte/estacionesSubte.csv", low_memory=False, encoding = 'ISO-8859-1')
estacionesTren = pd.read_csv("datosCompletos/transporte/estacionesTren.csv", low_memory=False,sep = ';')
def cargarLocalizacionesTransporte(df):
    df['cantidadSubtes'] = 0
    df['cantidadTrenes'] = 0
    cantidadEstacionesSubte = df.apply(lambda x:calcularEstaciones(x), axis = 1)
    df['cantidadSubtes'] = cantidadEstacionesSubte

In [17]:
#filleamos los nan con el promedio de las columnas
def fillNan(df):
    return df.apply(lambda x: x.fillna(x.mean()),axis=0)

In [18]:
dsSell = getMunicipioYZona(dsSell, 'train')
dsSell = filtrarPrecioNulo(dsSell)
formatearFecha(dsSell)

In [19]:
columna = 'municipio'
dfMetrosCuadradosPorZona = dsSell.groupby(columna).mean().reset_index()[[columna,'price_usd_per_m2']]
dsSell = getPrecioPorMetroCuadradoPorZona(dsSell, dfMetrosCuadradosPorZona, columna)

In [20]:
getInfoFromDescription(dsSell)
estandarizarCampoSuperfice(dsSell)



M:\Users\Krion\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
M:\Users\Krion\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [21]:
dropColumnas(dsSell)
preprocesamientoExpensas(dsSell)
dsSell = oneHotEncoding(dsSell, 'zona')
dsSell = oneHotEncoding(dsSell, 'property_type')
dsSell = fillNan(dsSell)
#cargarLocalizacionesTransporte(dsSell)

M:\Users\Krion\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)


In [22]:
#creamos un set de test y otro de entrenamiento. el test de entrenamiento sera de un 10% del original 

dsSell['is_true'] = np.random.uniform(0,1,len(dsSell)) >= 0.3
dsSellTrain, dsSellTest = dsSell[dsSell['is_true']==True], dsSell[dsSell['is_true']==False] 
del dsSell['is_true']
del dsSellTrain['is_true']
del dsSellTest['is_true']


dsSellTrain_y = dsSellTrain['price_aprox_usd']
dsSellTrain_x = dsSellTrain.drop('price_aprox_usd', axis=1)
dsSellTest_y = dsSellTest['price_aprox_usd']
dsSellTest_x = dsSellTest.drop('price_aprox_usd', axis=1)

In [23]:
columnas = dsSellTrain_x.columns.tolist()

In [24]:
def gridSearch(model, train_x, train_y):

SyntaxError: unexpected EOF while parsing (<ipython-input-24-fd0e7330b8dd>, line 1)

In [25]:
#normalize data
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model


#dsSellTrain_x = preprocessing.scale(dsSellTrain_x)
#dsSellTrain_x = preprocessing.normalize(dsSellTrain_x)
scaler = StandardScaler()
scaler.fit(dsSellTrain_x) 
dsSellTrain_x= scaler.transform(dsSellTrain_x)
dsSellTest_x = scaler.transform(dsSellTest_x)

In [71]:
from sklearn.neural_network import MLPRegressor
from sklearn import svm
from sklearn import linear_model
modelo =1  #0: Linear Regression, 1: MLP
             
if(modelo == 0):
    model = linear_model.LinearRegression()
if(modelo == 1):
    model = MLPRegressor(hidden_layer_sizes=(30,30,30,30), max_iter = 150)


In [72]:
import time

start = time.time()
print("[INFO] fitting stage")
model.fit(dsSellTrain_x,dsSellTrain_y)
print("[INFO] fitting state took {:.2f} seconds".format(time.time() - start))

[INFO] fitting stage
[INFO] fitting state took 667.83 seconds


In [73]:
predictions = model.predict(dsSellTest_x)

In [74]:
from sklearn.metrics import mean_squared_error
mean_squared_error(dsSellTest_y, predictions)

93342649312.408096

In [ ]:
#usar SGD_Regressor
if (modelo == 0):
    params =     { 'loss' : ['squared_epsilon_insensitive'],
          'penalty': ['l1', 'l2', 'elasticnet'] ,'learning_rate' : ['invscaling', 'optimal'] , 'eta0' : [1,0.1,0.01,0.0001,0.0000001,10,100,1000,0.5,5,1.22,1.6,7,9,20,0.3,0.05,], 'power_t' : [0.001,0.1,0.45,0.01,0.6],
          'epsilon' : [0.8,0.85,0.1,0.5,0.2], 'l1_ratio' : [0.01,0.2,0.4,0.5,0.6,0.8],
             'alpha' : [1,0.1,0.01,0.001,0.0001,0.0001,0.00001,0.5,5,0.2,0.006,0.3,0.7,.003,0.04]}  

  
    model = linear_model.SGDRegressor()

In [49]:
#random search del modelo elegido
from sklearn.model_selection import RandomizedSearchCV
from numpy import random

from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import make_scorer

import scipy


print("[INFO] tuning hyperparameters via grid search")


grid = RandomizedSearchCV(model, params, n_jobs = 5, n_iter =100)


start = time.time()
print("[INFO] fitting stage")
grid = grid.fit(dsSellTrain_x , dsSellTrain_y)
acc = grid.best_score_
model.get_params()
print("[INFO] grid search took {:.2f} seconds".format(time.time() - start))
print("[INFO] grid search accuracy: {:.2f}%".format(acc * 100))
print("[INFO] grid search best parameters: {}".format(grid.best_params_))

[INFO] tuning hyperparameters via grid search
[INFO] fitting stage
[INFO] grid search took 241.75 seconds
[INFO] grid search accuracy: 8.97%
[INFO] grid search best parameters: {'power_t': 0.1, 'penalty': 'elasticnet', 'loss': 'squared_epsilon_insensitive', 'learning_rate': 'optimal', 'l1_ratio': 0.8, 'eta0': 0.01, 'epsilon': 0.5, 'alpha': 5}


In [75]:
#levantamos el set de test 
testSet = pd.read_csv("testSet.csv", low_memory=False)
testId = testSet['id']
testSet

,id,created_on,property_type,operation,place_name,place_with_parent_names,country_name,state_name,lat-lon,lat,lon,surface_total_in_m2,surface_covered_in_m2,floor,rooms,expenses,description
0,3632,2017-08-24,departamento,venta,Puerto Madero,|Argentina|Capital Federal|Puerto Madero|,Argentina,Capital Federal,"-34.6109877599,-58.3634635778",-34.610988,-58.363464,0.0,NaN,NaN,NaN,NaN,Edificio BA Houses situado frente al Dique 3 d...
1,3633,2017-08-25,departamento,venta,Buenos Aires Interior,|Argentina|Buenos Aires Interior|,Argentina,Buenos Aires Interior,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,El departamento cuenta con un living-comedor a...
2,2263404,2017-08-01,departamento,venta,Palermo Soho,|Argentina|Capital Federal|Palermo|Palermo Soho|,Argentina,Capital Federal,"-34.5893633232,-58.4128798588",-34.589363,-58.412880,53.0,48.0,NaN,NaN,1500,IMPECABLE TORRE COY III – DEPA...
3,2263405,2017-08-01,departamento,venta,Chacarita,|Argentina|Capital Federal|Chacarita|,Argentina,Capital Federal,NaN,NaN,NaN,39.0,39.0,NaN,NaN,NaN,AMBIENTE DIVISIBLE CON PISOS D...
4,2263406,2017-08-01,departamento,venta,Chacarita,|Argentina|Capital Federal|Chacarita|,Argentina,Capital Federal,NaN,NaN,NaN,51.0,51.0,NaN,NaN,NaN,LIVING COMEDOR CON PISOS DE PO...
5,2263407,2017-08-01,departamento,venta,Chacarita,|Argentina|Capital Federal|Chacarita|,Argentina,Capital Federal,NaN,NaN,NaN,39.0,39.0,NaN,NaN,NaN,AMBIENTE DIVISIBLE CON PISOS D...
6,2263408,2017-08-01,departamento,venta,Chacarita,|Argentina|Capital Federal|Chacarita|,Argentina,Capital Federal,NaN,NaN,NaN,39.0,39.0,NaN,NaN,NaN,AMBIENTE DIVISIBLE CON PISOS D...
7,2263409,2017-08-01,departamento,venta,Chacarita,|Argentina|Capital Federal|Chacarita|,Argentina,Capital Federal,NaN,NaN,NaN,39.0,39.0,NaN,NaN,NaN,AMBIENTE DIVISIBLE CON PISOS D...
8,2263410,2017-08-01,departamento,venta,Colegiales,|Argentina|Capital Federal|Colegiales|,Argentina,Capital Federal,"-34.5787213694,-58.4570952614",-34.578721,-58.457095,67.0,67.0,NaN,NaN,2600,EXCELENTE 2 AMB- APTO PROFESIO...
9,2263411,2017-08-01,departamento,venta,Almagro,|Argentina|Capital Federal|Almagro|,Argentina,Capital Federal,NaN,NaN,NaN,47.0,47.0,NaN,NaN,NaN,IMPECABLE MONOAMBIENTE- DIVISI...


In [76]:
#drop de columnas
def dropColumnasTestSet(df):
    df.drop('place_with_parent_names', axis = 1, inplace = True)
    df.drop('created_on', axis = 1, inplace = True)
    df.drop('description', axis = 1, inplace = True)
    df.drop('id', axis=1, inplace=True)
    df.drop('lat-lon', axis=1, inplace=True)
    df.drop('operation', axis=1, inplace=True)
    df.drop('country_name', axis=1, inplace=True)
    df.drop('place_name', axis=1, inplace=True)
    df.drop('municipio', axis=1, inplace=True)
    df.drop('state_name', axis=1, inplace=True)
    df.drop('filtro', axis=1, inplace=True)

In [77]:
#preprocesamos el set de test
testSetPreprocessed = testSet

testSetPreprocessed = getMunicipioYZona(testSetPreprocessed, 'test')
formatearFecha(testSetPreprocessed)
getInfoFromDescription(testSetPreprocessed)
dropColumnasTestSet(testSetPreprocessed)
preprocesamientoExpensas(testSetPreprocessed)
testSetPreprocessed = pd.merge(testSetPreprocessed ,  dfMetrosCuadradosPorZona, on=columna, how = 'left')
testSetPreprocessed = oneHotEncoding(testSetPreprocessed, 'zona')
testSetPreprocessed = oneHotEncoding(testSetPreprocessed, 'property_type')
testSetPreprocessed = fillNan(testSetPreprocessed)

M:\Users\Krion\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)


KeyError: 'municipio'

In [78]:
columnas

['expenses',
 'lat',
 'lon',
 'rooms',
 'surface_covered_in_m2',
 'surface_total_in_m2',
 'year',
 'month',
 'price_usd_per_m2',
 'cochera',
 'pileta',
 'sum',
 'aire acondicionado',
 'calefaccion',
 'seguridad',
 'estrenar',
 'emprendimiento',
 'Bs.As. G.B.A. Zona Norte',
 'Bs.As. G.B.A. Zona Oeste',
 'Bs.As. G.B.A. Zona Sur',
 'Capital Federal',
 'PH',
 'apartment',
 'house',
 'store']

In [122]:
testSetPreprocessed.columns

Index(['lat', 'lon', 'surface_total_in_m2', 'surface_covered_in_m2', 'floor',
       'rooms', 'expenses', 'year', 'month', 'cochera', 'pileta', 'sum',
       'aire acondicionado', 'calefaccion', 'seguridad', 'estrenar',
       'emprendimiento', 'price_usd_per_m2', 'Bs.As. G.B.A. Zona Norte',
       'Bs.As. G.B.A. Zona Oeste', 'Bs.As. G.B.A. Zona Sur',
       'Buenos Aires Interior', 'Capital Federal', 'casa', 'departamento',
       'ph'],
      dtype='object')

In [123]:
testSetPreprocessed.rename(columns={'ph': 'PH', 'departamento': 'apartment',  'casa': 'house'}, inplace=True)
testSetPreprocessed['store'] = 0

In [124]:
testSetPreprocessed = testSetPreprocessed[columnas]  

In [125]:
testSetPreprocessed = scaler.transform(testSetPreprocessed)

In [127]:
testPrice= model.predict(testSetPreprocessed)

In [128]:
dataPredictions = {'id' : testId,
        'price_usd' : testPrice}
testPrediction = pd.DataFrame(dataPredictions)

In [129]:
testPrediction

,id,price_usd
0,3632,2.567337e+05
1,3633,1.328486e+17
2,2263404,2.607017e+05
3,2263405,2.587817e+05
4,2263406,2.594857e+05
5,2263407,2.587817e+05
6,2263408,2.587817e+05
7,2263409,2.587817e+05
8,2263410,2.614697e+05
9,2263411,2.592297e+05


In [131]:
#exportar csv con formato para kaggle
fileName = "pricePrediction.csv"
testPrediction.to_csv(fileName, sep=',', index = False)